In [ ]:
"""
Routing Pattern in Agentic AI: Customer Support Use Case
========================================================

This example demonstrates how to route customer queries to specialized
AI agents based on the query type. This is a fundamental pattern in
building agentic AI systems.
"""

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnableLambda
from langchain_openai import ChatOpenAI
from typing import Dict
from dotenv import load_dotenv
load_dotenv()

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ============================================================================
# Step 1: Create the Router Agent
# ============================================================================

router_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a query classifier. Classify customer queries into one category:
- order_status: Order tracking, delivery, shipping, returns, refunds
- product_info: Product features, specs, availability, pricing
- tech_support: Technical issues, troubleshooting, bugs, errors

Respond with ONLY the category name."""),
    ("user", "{query}")
])

#uses LCEL
router_chain = router_prompt | llm | StrOutputParser()

# ============================================================================
# Step 2: Create Specialized Agent Chains
# ============================================================================

# Order Status Agent
order_agent = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful order support agent. Help customers with order tracking, delivery status, returns, and shipping questions. Be friendly and professional."),
    ("user", "{query}")
]) | llm | StrOutputParser()

# Product Information Agent
product_agent = ChatPromptTemplate.from_messages([
    ("system", "You are a product specialist. Provide accurate information about product features, specifications, availability, and pricing."),
    ("user", "{query}")
]) | llm | StrOutputParser()

# Technical Support Agent
tech_agent = ChatPromptTemplate.from_messages([
    ("system", "You are a technical support specialist. Help troubleshoot issues, provide step-by-step solutions, and explain technical problems clearly."),
    ("user", "{query}")
]) | llm | StrOutputParser()

# ============================================================================
# Step 3: Implement the Routing Logic using RunnableBranch
# ============================================================================

def classify_and_route(input_dict: Dict) -> Dict:
    """Classify query and add category to input for routing."""
    query = input_dict["query"]
    category = router_chain.invoke({"query": query}).strip().lower()
    input_dict["category"] = category
    return input_dict

# Helper to extract query for agents
def extract_query(x: Dict) -> Dict:
    """Extract query from input dict for agent chains."""
    return {"query": x["query"]}

# Create the branch router using RunnableBranch
branch_router = RunnableBranch(
    (lambda x: "order" in x.get("category", "").lower(), 
     RunnableLambda(extract_query) | order_agent),
    (lambda x: "product" in x.get("category", "").lower(), 
     RunnableLambda(extract_query) | product_agent),
    (lambda x: "tech" in x.get("category", "").lower() or "support" in x.get("category", "").lower(), 
     RunnableLambda(extract_query) | tech_agent),
    RunnableLambda(extract_query) | tech_agent  # default chain
)

# Combine classification and routing
routing_pipeline = RunnableLambda(classify_and_route) | branch_router

def route_with_metadata(query: str) -> Dict:
    """Route query using RunnableBranch and return response with metadata."""
    category = router_chain.invoke({"query": query}).strip().lower()
    response = routing_pipeline.invoke({"query": query})
    
    return {
        "query": query,
        "category": category,
        "response": response
    }
# ============================================================================
# Step 4: Test the Routing System
# ============================================================================

result = route_with_metadata("My laptop smells like burning plastic")
print(f"Category: {result['category']}")
print(f"Response: {result['response']}")

Category: tech_support
Response: A burning plastic smell coming from your laptop can be concerning and may indicate a serious issue. Here are some steps to troubleshoot the problem:

### Step 1: Power Off and Unplug
1. **Immediately turn off your laptop**: Press and hold the power button until it shuts down.
2. **Unplug the power adapter**: Disconnect the charger from the laptop and the wall outlet.

### Step 2: Allow to Cool
1. **Let the laptop cool down**: Place it on a hard, flat surface and allow it to cool for at least 30 minutes. This will help prevent any further damage or risk of fire.

### Step 3: Inspect the Laptop
1. **Check for visible damage**: Look for any signs of melting, burning, or discoloration on the laptop's casing, especially around the vents, ports, and battery area.
2. **Listen for unusual sounds**: If you hear any unusual sounds (like buzzing or grinding), this could indicate a hardware failure.

### Step 4: Check the Battery
1. **Remove the battery (if possibl